In [1]:
import pandas as pd

path = '/data/home/zhaocj/spn_forcast/df数据整理/../7mer_output/RBIND_7mer'

df = pd.read_table(path, header=None, names=['SEQ', 'COUNT', 'ID'])
id_number = pd.read_table('/data/home/zhaocj/spn_forcast/df数据整理/ID.txt')
mic = pd.read_table('/data/home/zhaocj/spn_forcast/df数据整理/mic.txt')
ris = pd.read_table('/data/home/zhaocj/spn_forcast/df数据整理/ris.txt')
sero = pd.read_table('/data/home/zhaocj/seroba/summary.tsv', header=None, names=['ID', 'SEROTYPE', 'COMMENT'])
sero2 = pd.read_table('sero2.txt')
sero2['COMMENT'] = 'NaN'
sero = pd.concat([sero, sero2])
sero.rename(columns={'SEROTYPE':'SEROTYPE_AI'}, inplace = True)
mlst = pd.read_table('MLST.txt')

/data/build/miniconda3/envs/cenote-taker2_env/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3072: DtypeWarning: Columns (2) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [2]:
df = df.pivot_table(index=['ID'], columns='SEQ', values='COUNT', fill_value=0).reset_index()

# 把 df 的ID转换为字符型
for i in range(df.shape[0]):
    df.iloc[i, 0] = str(df.iloc[i, 0])

# 选取id_number数据表里有的菌
df = df.set_index('ID')
df = df.loc[list(id_number['ID'])].reset_index()

df = df[['ID']]

# 合并MIC
df = pd.merge(df, mic, how = 'left', on=['ID'])

# 合并手工血清型
df = pd.merge(df, ris[['ID', 'SEROTYPE']], how = 'left', on=['ID'])

# 合并seroBA血清型
df = pd.merge(df, sero, how = 'left', on=['ID'])

# 合并MLST
df = pd.merge(df, mlst, how = 'left', on=['ID'])

In [3]:
# 看一眼
df

,ID,PATIENT_ID,ORGANISM,PEN_MIC,AMC_MIC,CRO_MIC,ERY_MIC,CLI_MIC,LVX_MIC,MFX_MIC,SXT_MIC,SEROTYPE,SEROTYPE_AI,COMMENT,ST
0,100,HB-19-04,spn,0.125,0.25,0.25,16.0,16.00,1.0,0.125,2.00,6A,06A,NaN,15667
1,101,HB-19-05,spn,0.125,0.25,0.25,16.0,0.25,1.0,0.125,2.00,23F,23F,NaN,13646
2,103,HB-19-07,spn,2.000,1.00,0.25,8.0,0.25,0.5,0.125,1.00,19F,06B,NaN,5244
3,104,HB-19-08,spn,1.000,0.50,0.25,16.0,16.00,1.0,0.125,0.25,19F,19F,NaN,236
4,105,HB-19-09,spn,1.000,0.50,0.25,16.0,16.00,1.0,0.125,4.00,19F,19F,NaN,236
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
628,X96,TS-19-94,spn,4.000,8.00,8.00,16.0,16.00,1.0,0.125,16.00,19F,19F,NaN,271
629,X97,TS-19-95,spn,4.000,8.00,4.00,16.0,16.00,1.0,0.125,16.00,19F,19F,NaN,271
630,X98,TS-19-96,spn,0.125,0.25,0.25,16.0,16.00,1.0,0.125,4.00,9N,09V,contamination,1263
631,X99,BR-19-1,spn,2.000,1.00,4.00,16.0,16.00,1.0,0.125,0.50,14,14,NaN,876


In [5]:
# 导出数据
df.to_csv('633菌株MIC.txt')